<a href="https://colab.research.google.com/github/joshuaalpuerto/ML-guide/blob/main/Eval_STT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U evaluate
!pip install rouge_score
!pip install jiwer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 27.2 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 10.0.1
    Uninstalling pyarrow-10.0.1:
      Successfully uninstalled pyarrow-10.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibis-framework 7.1.0 requires pyarrow<15,>=2, but you have pyarrow 15.0.0 which is incompatible.
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=ea05c1bd22396c6ae3aeb86

In [ ]:
import evaluate
wer = evaluate.load('wer')

predictions = ["hi how are you today John Super nice!"]
references = ["Hi how are you today John, Super nice!"]

results = wer.compute(predictions=predictions,
                     references=references)

print(results)

0.25


In [ ]:
from jiwer import compute_measures


reference = ["Hi how are you today John, Super nice!"]
hypothesis = ["hi how are you today John Super nice!"]

error = compute_measures(reference, hypothesis)

[[AlignmentChunk(type='substitute', ref_start_idx=0, ref_end_idx=1, hyp_start_idx=0, hyp_end_idx=1), AlignmentChunk(type='equal', ref_start_idx=1, ref_end_idx=5, hyp_start_idx=1, hyp_end_idx=5), AlignmentChunk(type='substitute', ref_start_idx=5, ref_end_idx=6, hyp_start_idx=5, hyp_end_idx=6), AlignmentChunk(type='equal', ref_start_idx=6, ref_end_idx=8, hyp_start_idx=6, hyp_end_idx=8)]]


In [ ]:
import pandas as pd

# Existing DataFrame
df = pd.DataFrame(columns=['reference', 'substitutions', 'insertions', 'deletions'])


def extract_incorrect_hypothesis(current_hypothesis, word_eval):
  return current_hypothesis[word_eval.hyp_start_idx:word_eval.hyp_end_idx][0]

for idx, op in enumerate(error['ops']):
  current_hypothesis = error['hypothesis'][idx]
  new_df = {
    'reference': reference[idx],
    'substitutions': [extract_incorrect_hypothesis(current_hypothesis, word_eval) for word_eval in op if word_eval.type == 'substitute'],
    'insertions': [extract_incorrect_hypothesis(current_hypothesis, word_eval) for word_eval in op if word_eval.type == 'insert'],
    'deletions': [extract_incorrect_hypothesis(current_hypothesis, word_eval) for word_eval in op if word_eval.type == 'delete']
  }
  df = df.append(new_df, ignore_index=True)


df.head(5)

<ipython-input-33-4bf15cdd3883>:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_df, ignore_index=True)


,reference,substitutions,insertions,deletions
0,"Hi how are you today John, Super nice!","[hi, John]",[],[]


In [ ]:
#connect to google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

# creating a data frame
df = pd.read_csv("/content/drive/MyDrive/datasets/stt-data.csv")

In [ ]:
import evaluate

rouge = evaluate.load('rouge')
predictions = df['web_speech']
references = df['label']
results = rouge.compute(predictions=predictions,
                     references=references)

print(results)

{'rouge1': 0.7734969534114653, 'rouge2': 0.7043393166236724, 'rougeL': 0.7642496273048476, 'rougeLsum': 0.7636199619962072}


# Rouge metrics

NOTE: That rouge metric doesn't consider punctation also NOT case-sensitive.

In [ ]:
import evaluate
rouge = evaluate.load('rouge')

def compute_rouge_metric(labels, preds):
  return rouge.compute(predictions=preds,
                     references=labels)

In [ ]:
import pandas as pd

# creating a data frame
df = pd.read_csv("/content/drive/MyDrive/datasets/stt-data-prod.csv")
filtered_csb_df = df[df['label'].notna() & df['csb_slow'].notna() & df['csb_fast'].notna()].reset_index(drop=True)
filtered_dg = df[df['label'].notna() & df['deepgram'].notna()].reset_index(drop=True)



csb_slow_rouge_metric = compute_rouge_metric(filtered_csb_df['label'], filtered_csb_df['csb_slow'])
csb_fast_rouge_metric = compute_rouge_metric(filtered_csb_df['label'], filtered_csb_df['csb_fast'])
deepgram_rouge_metric = compute_rouge_metric(filtered_dg['label'], filtered_dg['deepgram'])

df = pd.DataFrame({
    'CSB_SLOW': csb_slow_rouge_metric,
    'CSB_FAST': csb_fast_rouge_metric,
    'DEEPGRAM': deepgram_rouge_metric,
})


df.head(10)

,CSB_SLOW,CSB_FAST,DEEPGRAM
rouge1,0.780757,0.760371,0.903800
rouge2,0.686435,0.658419,0.819910
rougeL,0.776524,0.753866,0.899960
rougeLsum,0.777798,0.753058,0.900344


# WER Metrics

NOTE: The WER is defined such that lower is better: a lower WER means there are fewer errors in our prediction, so a perfect speech recognition system would have a WER of zero (no errors).

More info: https://huggingface.co/learn/audio-course/en/chapter5/evaluation#evaluation-metrics-for-asr

In [ ]:
import evaluate
wer = evaluate.load('wer')

def compute_wer_metric(labels, preds):
  return wer.compute(predictions=preds,
                     references=labels)

In [ ]:
import pandas as pd

# creating a data frame
df = pd.read_csv("/content/drive/MyDrive/datasets/stt-data-prod.csv")
filtered_csb_df = df[df['label'].notna() & df['csb_slow'].notna() & df['csb_fast'].notna()].reset_index(drop=True)
filtered_dg = df[df['label'].notna() & df['deepgram'].notna()].reset_index(drop=True)



csb_slow_wer_metrics = compute_wer_metric(filtered_csb_df['label'], filtered_csb_df['csb_slow'])
csb_fast_wer_metrics = compute_wer_metric(filtered_csb_df['label'], filtered_csb_df['csb_fast'])
deepgram_wer_metrics = compute_wer_metric(filtered_dg['label'], filtered_dg['deepgram'])


df = pd.DataFrame({
    'CSB_SLOW': { "score": csb_slow_wer_metrics},
    'CSB_FAST': { "score": csb_fast_wer_metrics},
    'DEEPGRAM': { "score": deepgram_wer_metrics},
})



df.head(10)

,CSB_SLOW,CSB_FAST,DEEPGRAM
score,0.419355,0.471486,0.291139


In [ ]:
import pandas as pd
from jiwer import compute_measures

truth = list(filtered_dg['label'])
hypothesis=list(filtered_dg['deepgram'])


results = compute_measures(truth, hypothesis)

# Existing DataFrame
df = pd.DataFrame(columns=['label', 'prediction', 'substitutions', 'insertions', 'deletions'])


def extract_incorrect_hypothesis(current_hypothesis, word_eval):
  result =  current_hypothesis[word_eval.hyp_start_idx:word_eval.hyp_end_idx]
  return "|".join(result)

for idx, op in enumerate(results['ops']):
  current_hypothesis = results['hypothesis'][idx]

  new_df = {
    'label': truth[idx],
    'prediction': hypothesis[idx],
    'substitutions': [extract_incorrect_hypothesis(current_hypothesis, word_eval) for word_eval in op if word_eval.type == 'substitute'],
    'insertions': [extract_incorrect_hypothesis(current_hypothesis, word_eval) for word_eval in op if word_eval.type == 'insert'],
    'deletions': [extract_incorrect_hypothesis(current_hypothesis, word_eval) for word_eval in op if word_eval.type == 'delete']
  }
  df = df.append(new_df, ignore_index=True)


df.head(50)